# Example of using DOpt Federov Exchange Algorithm
## Algorithm obtained from
-  **Algorithm AS 295:** A Fedorov Exchange Algorithm for D-Optimal Design
-  **Author(s):** Alan J. Miller and Nam-Ky Nguyen
-  **Source:** Journal of the Royal Statistical Society. Series C (Applied Statistics), Vol. 43, No. 4, pp. 669-677, 1994
-  **Stable URL:** http://www.jstor.org/stable/2986264

## Source code from
-  http://ftp.uni-bayreuth.de/math/statlib/apstat/

## Notes
- This is a two design variable, kwadratic model example problem from Myers and Montgomery, Response Surface Methodology

# Load the dopt shared library that provides the interface

### Print the documentation and note that
### Input
-  $x$ is the 2D numpy array that contains the candidate points to select from
-  $n$ is the number of points in the final design
-  $in$ is the number of preselected points that MUST be in the final design (>= 0)
-  $rstart$ indicate if a random start should be performed, should be True in most cases.  If False the user must supply the initial design in $picked$
-  $picked$ is a 1D array that contains the preselected point ID's (remember FORTRAN is 1 based array) on input.  The first $in$ entries are read for ID's.  On output it contains the ID's in x of the final selection

### Output
-  $lndet$ is the logarithm of the determinant of the best design
-  $ifault$ is possible fault codes
>-  -1 if no full rank starting design is found
>-  0 if no error is detected
>-  1* if DIM1 < NCAND
>-  2* if K < N
>-  4* if NRBAR < K(K - 1)/2
>-  8* if K KIN + NBLOCK
>-  16* if the sum of block sizes is not equal to N
>-  32* if any IN(I) < 0 or any IN(I) > BLKSIZ(I)

In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import math as m
import dopt
print( dopt.dopt.__doc__ )

lndet,ifault = dopt(x,n,in,rstart,picked)

Wrapper for ``dopt``.

Parameters
----------
x : input rank-2 array('d') with bounds (dim1,kin)
n : input int
in : input int
rstart : input int
picked : input rank-1 array('i') with bounds (n)

Returns
-------
lndet : float
ifault : int



# Load the sample data set from the Excel spreadsheet and clean it up by removing all duplicate points
>-  2 Design variables, Full Quadratic model

In [2]:
# Sample data set from Excel spreadsheet
filename = 'MyersExample.xlsx'
xls = pd.ExcelFile(filename)
df1 = pd.read_excel(xls, 'Sheet2')

# Remove all duplicate rows from the data set - Note that the dataset now only have 8 unique values
df1 = df1.drop_duplicates(subset=['x1','x2'], keep='first')
print(df1)

# Pull out the 3 and 4th columns as the x1 and x2 variables that we will use to create the model matrix from
y  = df1.iloc[:, 5].values
x1 = df1.iloc[:, 3].values
x2 = df1.iloc[:, 4].values

# Scale the variables - Seems to work if we scale or not - is typically always a good idea to scale
x1 = (x1 + x1 - x1.min() - x1.max()) / (x1.max()-x1.min())
x2 = (x2 + x2 - x2.min() - x2.max()) / (x2.max()-x2.min())

# Setup the design matrix
x = np.zeros((len(x1), 6), float)
x[:,0] = 1.
x[:,1] = x1
x[:,2] = x2
x[:,3] = x1*x1
x[:,4] = x2*x2
x[:,5] = x1*x2
print(' ')
print (x)

   Observation      z1     z2     x1     x2   y
0            1  200.00  15.00 -1.000 -1.000  43
1            2  250.00  15.00  1.000 -1.000  78
2            3  200.00  25.00 -1.000  1.000  69
3            4  250.00  25.00  1.000  1.000  73
4            5  189.65  20.00 -1.414  0.000  48
5            6  260.35  20.00  1.414  0.000  78
6            7  225.00  12.93  0.000 -1.414  65
7            8  225.00  27.07  0.000  1.414  74
8            9  225.00  20.00  0.000  0.000  76
 
[[ 1.         -0.70721358 -0.70721358  0.50015105  0.50015105  0.50015105]
 [ 1.          0.70721358 -0.70721358  0.50015105  0.50015105 -0.50015105]
 [ 1.         -0.70721358  0.70721358  0.50015105  0.50015105 -0.50015105]
 [ 1.          0.70721358  0.70721358  0.50015105  0.50015105  0.50015105]
 [ 1.         -1.          0.          1.          0.         -0.        ]
 [ 1.          1.          0.          1.          0.          0.        ]
 [ 1.          0.         -1.          0.          1.         -0.   

# Call the interface and print the output and the picked array
-  We raise an exception with iFault is not 0 - this is just good practice
-  We repeat the DOptimal process 10 times and pick the best design.  We do this in an attempt to avoid local minima

In [3]:
# Number of points to pick - we can pick a max of 9 and a minimum of 6
n = 8

# Array of point ID's that will be picked
picked = np.zeros( n, np.int32 )

# Number of picked points (points to force into the design)
npicked = 0

# Store the best design and the corresponding determinant values
bestDes = np.copy( picked )
bestDet = 0
rstart  = True   # Look at documentation for 295 - should not really need to change

# Repeat the process 10 times and store the best design
for i in range(0, 10) :
    
    # Make the DOptimal call
    lnDet, iFault = dopt.dopt( x, n, npicked, rstart, picked)
    
    # Raise an exception if iFault is not equal to 0
    if iFault != 0:
        raise ValueError( "Non-zero return code form dopt algorith.  iFault = ", iFault )
       
    # Store the best design
    if m.fabs(lnDet) > bestDet:
        bestDet =lnDet
        bestDes = np.copy( picked )

# Print the best design out
print( "Maximum Determinant Found:", m.exp(bestDet) )
print( "\nBest Design Found (indices):\n", np.sort(bestDes) )
print( "\nBest Design Found (variables):\n", x[np.sort(bestDes)-1,1:4] )

Maximum Determinant Found: 48.07337205914986

Best Design Found (indices):
 [1 2 3 4 5 6 7 9]

Best Design Found (variables):
 [[-0.70721358 -0.70721358  0.50015105]
 [ 0.70721358 -0.70721358  0.50015105]
 [-0.70721358  0.70721358  0.50015105]
 [ 0.70721358  0.70721358  0.50015105]
 [-1.          0.          1.        ]
 [ 1.          0.          1.        ]
 [ 0.         -1.          0.        ]
 [ 0.          0.          0.        ]]


# Now solve the least squares problem

In [4]:
# Extract the DOptimum x and y values
y_opt = y[np.sort(bestDes)-1]
x_opt = x[np.sort(bestDes)-1]

# Setup the Statsmodels model and perform the fit
modOLS = sm.OLS( y_opt, x_opt )
resOLS = modOLS.fit()

# Print the summary of the ordinary least squares fit
print( resOLS.summary() )

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.995
Model:                            OLS   Adj. R-squared:                  0.982
Method:                 Least Squares   F-statistic:                     78.49
Date:                Tue, 03 Sep 2019   Prob (F-statistic):             0.0126
Time:                        14:10:00   Log-Likelihood:                -10.575
No. Observations:                   8   AIC:                             33.15
Df Residuals:                       2   BIC:                             33.63
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         76.0008      1.815     41.871      0.0

/usr/lib64/python3.6/site-packages/scipy/stats/stats.py:1416: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=8
  "anyway, n=%i" % int(n))
